In [1]:
import pandas as pd
import requests
import os
from scipy import misc
from skimage.metrics import structural_similarity as ssim
import cv2

In [2]:
df = pd.read_csv('./data.csv', sep=";")
# Rename columns using a dictionary
df = df.rename(columns={'ams_id_ams': 'Advertisement ID', 'ams_rep': 'Reporter Count', 'ams_nom_img': 'Image Name', 'ams_site': "Site", 'ams_url_annonce': 'Advertisement URL', 'ams_num_enreg_txt': 'Registration Number'})

# Alternatively, you can rename columns directly by assigning new column names to the 'columns' attribute
df.columns = ['Advertisement ID', 'Reporter Count', 'Image Name','Site', 'Advertisement URL', 'Registration Number']

In [3]:
df

,Advertisement ID,Reporter Count,Image Name,Site,Advertisement URL,Registration Number
0,2285451,52727304,0ef4fc26-098d-43cf-8b63-47d58e8b2ae9.jpg,Airbnb,https://www.airbnb.fr/rooms/52727304,7511100521109
1,2285451,52727304,dd4a5dac-9991-4a6d-ade3-55b123c41a58.jpg,Airbnb,https://www.airbnb.fr/rooms/52727304,7511100521109
2,2285451,52727304,8a18b5b2-6227-4bcf-bfa9-8df18490a3f9.jpg,Airbnb,https://www.airbnb.fr/rooms/52727304,7511100521109
3,2285451,52727304,2300932e-a3c9-4c8a-8d9a-9d388fa2dab6.jpg,Airbnb,https://www.airbnb.fr/rooms/52727304,7511100521109
4,2285451,52727304,7301ce2f-9692-48dd-978a-40c3283a50c0.jpg,Airbnb,https://www.airbnb.fr/rooms/52727304,7511100521109
...,...,...,...,...,...,...
1067,1988121,43922239,11296f90-0b8c-4d73-b8cd-e36010bcc4ac.jpg,Airbnb,https://www.airbnb.fr/rooms/43922239,7510201886086
1068,1988121,43922239,53f61a4b-98c7-4373-a16e-7af35865d5ea.jpg,Airbnb,https://www.airbnb.fr/rooms/43922239,7510201886086
1069,1988121,43922239,6061a287-03e5-4e38-bdf7-6e85837ed25e.jpg,Airbnb,https://www.airbnb.fr/rooms/43922239,7510201886086
1070,1988121,43922239,4ee49bca-0a38-4e76-8da3-5823450ba9d2.jpg,Airbnb,https://www.airbnb.fr/rooms/43922239,7510201886086


In [4]:
len(df['Advertisement ID'].unique())

45

In [5]:
df = df.drop(df[df['Registration Number'].isnull()].index)

In [6]:
pd.options.display.max_columns = None
print(df.isnull().sum())

Advertisement ID       0
Reporter Count         0
Image Name             0
Site                   0
Advertisement URL      0
Registration Number    0
dtype: int64


In [7]:
df = df.drop(df[df['Registration Number'] == 'Disponible uniquement via un bail mobilité'].index)

In [8]:
df = df.drop(df[df['Registration Number'] == 'Exemption : logement de type hôtelier'].index)

## Similarity Search

In [163]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer
from keras.applications.xception import Xception,preprocess_input
import tensorflow as tf
from keras.preprocessing import image
from keras.layers import Input
from keras.backend import reshape
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


## 2. Loading and Preprocessing of Data

In [10]:
images_dir = './Nouveau dossier/'

In [11]:
def getImagePaths(path):
    """
    Function to Combine Directory Path with individual Image Paths
    
    parameters: path(string) - Path of directory
    returns: image_names(string) - Full Image Path
    """
    image_names = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            fullpath = os.path.join(dirname, filename)
            image_names.append(fullpath)
    return image_names

def preprocess_img(img_path):
    dsize = (225,225)
    new_image=cv2.imread(img_path)
    new_image=cv2.resize(new_image,dsize,interpolation=cv2.INTER_NEAREST)  
    new_image=np.expand_dims(new_image,axis=0)
    new_image=preprocess_input(new_image)
    return new_image

def load_data():
    output=[]
    output=getImagePaths(images_dir)[:10000]
    return output

In [12]:
len(load_data())

1008


## 3. Defining model and extracting feature for all the training data


In [13]:
def model():
    model=Xception(weights='imagenet',include_top=False)
    for layer in model.layers:
        layer.trainable=False
        #model.summary()
    return model

def feature_extraction(image_data,model):
    features = model.predict(image_data)
    features = np.array(features)
    features = features.flatten()
    return features

In [14]:
def result_vector_cosine(model, feature_vector, new_img):
    new_feature = model.predict(new_img)
    new_feature = np.array(new_feature)
    new_feature = new_feature.flatten()
    N_result = 12
    nbrs = NearestNeighbors(n_neighbors=N_result, metric="cosine").fit(feature_vector)
    distances, indices = nbrs.kneighbors([new_feature])
    
    # Calculate similarity scores
    similarity_scores = 1 - distances
    max_similarity = similarity_scores.max()
    similarity_scores_normalized = similarity_scores/max_similarity
    
    return similarity_scores_normalized, indices

In [15]:
def input_show(data):
    plt.title("Query Image")
    plt.imshow(data)

In [162]:
def bgr_to_rgb(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [74]:
def show_result(data,result):
    fig = plt.figure(figsize=(12,8))
    for i in range(0,12):
        index_result=result[0][i]
        plt.subplot(3,4,i+1)
        # Read the image using OpenCV in BGR format
        bgr_image = cv2.imread(data[index_result])
        # Convert BGR image to RGB
        rgb_image = bgr_to_rgb(bgr_image)
        plt.imshow(rgb_image)
    plt.show()

In [18]:
output=load_data()
main_model=model()
features=[]
    
#Limiting the data for training
for i in output[:1008]:
    new_img=preprocess_img(i)
    features.append(feature_extraction(new_img,main_model))
feature_vec = np.array(features)

1/1 [==============================] - 0s 132ms/step


1/1 [==============================] - 0s 183ms/step


1/1 [==============================] - 0s 167ms/step


1/1 [==============================] - 0s 177ms/step


In [120]:
threshold = 0.75
data_path='./Nouveau dossier/'

image_paths = [output[idx] for idx in indices[0]]
matching_df = pd.DataFrame(columns=['Reg No 1', 'Reg No 2', 'Ad Num 1', 'Ad Num 2','Image Number 1',"Image Number 2" ,'Similarity Score'])

for index, row in df.iterrows():
    output_path=f"{data_path}{row['Registration Number']}/{row['Advertisement ID']}/photo_{row['Image Name']}"
    try:
        distances, indices = result_vector_cosine(main_model,feature_vec,preprocess_img(output_path))
    except Exception as e:
        print(e)
        print(output_path)
        continue
    filtered_path = []
    for dist,index in zip(distances[0], indices[0]):
        if(dist>threshold):
            filtered_path.append((output[index],dist))
    
#     print(filtered_paths)
    for file_path, score in filtered_path:
        # Extract the file name from the path
        file_name = os.path.basename(file_path)
        # Remove the "photo_" prefix
        image_name = file_name.replace("photo_", "")
        # Extract the advertisement number from the image name
        advertisement_number = file_path.split("/")[3]
        # Extract the advertisement number from the image name
        reg_number = file_path.split("/")[2]
        
#         print("Advertisement Number 1:", row['Advertisement ID'])
#         print("Reg Number 1:", row['Registration Number'])
#         print("Image Name 1:", row['Image Name'])

#         print("Advertisement Number 2:", advertisement_number)
#         print("Reg Number 2:", reg_number)
#         print("Image Name 2:", image_name)
        
         # Add the matching image information to the DataFrame
        matching_df = matching_df.append({
            'Reg No 1': row['Registration Number'],
            'Reg No 2': reg_number,
            'Image Number 1': row['Image Name'] ,
            'Image Number 2': image_name ,
            'Ad Num 1': row['Advertisement ID'],
            'Ad Num 2': advertisement_number,
            'Similarity Score': score
        }, ignore_index=True)
#     input_show(bgr_to_rgb(cv2.imread(output_path)))
#     show_result(output, indices)

1/1 [==============================] - 0s 134ms/step


/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({
/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({


1/1 [==============================] - 0s 162ms/step


/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({
/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({


1/1 [==============================] - 0s 152ms/step


...

/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({
/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({
/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({


1/1 [==============================] - 0s 242ms/step


/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({
/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({
/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({


1/1 [==============================] - 0s 236ms/step


/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({
/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({
/tmp/ipykernel_59012/14450016.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_df = matching_df.append({


In [158]:
# Group by 'Ad Num 1' and 'Ad Num 2' and calculate the count of each combination
# count_df = matching_df.groupby(['Ad Num 1', 'Ad Num 2']).size().reset_index(name='Count')
# count_df = count_df[count_df['Count']>5]

# Display the count DataFrame
# print(count_df[count_df['Count']>])
# matching_df[matching_df[']
count_df = matching_df.groupby(['Ad Num 1', 'Ad Num 2','Reg No 1','Reg No 2']).size().reset_index(name='Count')

# Convert 'Ad Num 1' and 'Ad Num 2' to strings
count_df['Ad Num 1'] = count_df['Ad Num 1'].astype(str)
count_df['Ad Num 2'] = count_df['Ad Num 2'].astype(str)

# Filter rows based on 'Count' greater than 10 and 'Ad Num 1' is different from 'Ad Num 2'
filtered_count_df = count_df[(count_df['Ad Num 1'] != count_df['Ad Num 2'])]

# Display the filtered DataFrame
print(filtered_count_df)

   Ad Num 1 Ad Num 2       Reg No 1       Reg No 2  Count
1     63591  1010611  7510201886086  7510201886086     21
2     63591  1988121  7510201886086  7510201886086     16
5     81391   619191  7511100521109  7511100521109      2
9    225620  1904081  7511100521109  7511100521109     19
10   225620  2070091  7511100521109  7511100521109     20
12   230370  1900611  7511100521109  7511100521109     20
13   230370  2064251  7511100521109  7511100521109     20
15   283840  2064891  7511100521109  7511100521109     39
16   283840  2333801  7511100521109  7511100521109     21
26   619191    81391  7511100521109  7511100521109      2
27   908021  1294481  7511100521109  7511100521109     12
32  1010611  1988121  7510201886086  7510201886086     16
33  1010611    63591  7510201886086  7510201886086     21
35  1294481   908021  7511100521109  7511100521109     12
40  1900611  2064251  7511100521109  7511100521109     22
41  1900611   230370  7511100521109  7511100521109     20
43  1904081  2

In [160]:
import numpy as np

# Assuming you already have the DataFrame filtered_count_df as mentioned earlier
# and you want to remove the repeated rows where 'Ad Num 1' and 'Ad Num 2' are reversed but 'Count' is the same.

# Sort 'Ad Num 1' and 'Ad Num 2' within each row
filtered_count_df[['Ad Num 1', 'Ad Num 2']] = np.sort(filtered_count_df[['Ad Num 1', 'Ad Num 2']], axis=1)

# Drop duplicates based on the sorted values of 'Ad Num 1' and 'Ad Num 2'
filtered_count_df.drop_duplicates(subset=['Ad Num 1', 'Ad Num 2'], keep='first', inplace=True)

# Display the updated DataFrame
print(filtered_count_df)

# Save the updated DataFrame to a CSV file
filtered_count_df.to_csv('filtered_count_df.csv', index=False)

# Display a message to indicate successful saving
print("DataFrame saved to 'filtered_count_df.csv'")

   Ad Num 1 Ad Num 2       Reg No 1       Reg No 2  Count
1   1010611    63591  7510201886086  7510201886086     21
2   1988121    63591  7510201886086  7510201886086     16
5    619191    81391  7511100521109  7511100521109      2
9   1904081   225620  7511100521109  7511100521109     19
10  2070091   225620  7511100521109  7511100521109     20
12  1900611   230370  7511100521109  7511100521109     20
13  2064251   230370  7511100521109  7511100521109     20
15  2064891   283840  7511100521109  7511100521109     39
16  2333801   283840  7511100521109  7511100521109     21
27  1294481   908021  7511100521109  7511100521109     12
32  1010611  1988121  7510201886086  7510201886086     16
40  1900611  2064251  7511100521109  7511100521109     22
43  1904081  2070091  7511100521109  7511100521109     21
53  2064891  2333801  7511100521109  7511100521109     72
61  2261471  2285451  7511100521109  7511100521109      3
62  2261471  2286101  7511100521109  7511100521109      2
63  2261471  2

/tmp/ipykernel_59012/737551877.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_count_df[['Ad Num 1', 'Ad Num 2']] = np.sort(filtered_count_df[['Ad Num 1', 'Ad Num 2']], axis=1)
/tmp/ipykernel_59012/737551877.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_count_df.drop_duplicates(subset=['Ad Num 1', 'Ad Num 2'], keep='first', inplace=True)


In [122]:
match_df = matching_df

In [123]:
matching_df

,Reg No 1,Reg No 2,Ad Num 1,Ad Num 2,Image Number 1,Image Number 2,Similarity Score
0,7511100521109,7511100521109,2285451,2333731,0ef4fc26-098d-43cf-8b63-47d58e8b2ae9.jpg,c4854285-ad61-4ca7-8223-5f2963db5358.jpeg,1.000000
1,7511100521109,7511100521109,2285451,2285451,0ef4fc26-098d-43cf-8b63-47d58e8b2ae9.jpg,0ef4fc26-098d-43cf-8b63-47d58e8b2ae9.jpg,1.000000
2,7511100521109,7511100521109,2285451,2333731,dd4a5dac-9991-4a6d-ade3-55b123c41a58.jpg,94b33710-3bdc-4683-ab79-24b218ae8ae3.jpeg,1.000000
3,7511100521109,7511100521109,2285451,2285451,dd4a5dac-9991-4a6d-ade3-55b123c41a58.jpg,dd4a5dac-9991-4a6d-ade3-55b123c41a58.jpg,1.000000
4,7511100521109,7511100521109,2285451,2333731,8a18b5b2-6227-4bcf-bfa9-8df18490a3f9.jpg,51a11283-0ed7-44b9-acae-474b847f7cba.jpeg,1.000000
...,...,...,...,...,...,...,...
1947,7510201886086,7510201886086,1988121,1010611,4ee49bca-0a38-4e76-8da3-5823450ba9d2.jpg,050f0634-ca71-4ec1-a0f4-56ee5678fae0.jpg,0.967738
1948,7510201886086,7510201886086,1988121,63591,4ee49bca-0a38-4e76-8da3-5823450ba9d2.jpg,cc8b0315-31aa-4f78-bf27-929023e377ee.jpg,0.967738
1949,7510201886086,7510201886086,1988121,1988121,d80cce7c-3a0c-44e4-8b07-0a36381b21e2.jpg,d80cce7c-3a0c-44e4-8b07-0a36381b21e2.jpg,1.000000
1950,7510201886086,7510201886086,1988121,63591,d80cce7c-3a0c-44e4-8b07-0a36381b21e2.jpg,97b15d47-b8fb-4c0d-ba99-cf682f8027a3.jpg,0.971585
